In [1]:
import cf
import cfplot as cfp
import matplotlib.pyplot as plt
import glob
import h5py
import xarray as xr
import xesmf as xe
import pandas as pd
import numpy as np

basedir = '/gws/nopw/j04/mcs_prime/mmuetz/data/UM_sims/u-db150/init_test/'
#org_on_path = basedir + 'um_on/'
#org_off_path = basedir + 'um_off/'
#obs_path = basedir + 'gpm/'
#cmip_path = basedir + 'pr_cmip6/'

In [2]:
%matplotlib

Using matplotlib backend: <object object at 0x7fb2fc55f1e0>


In [3]:
import calendar
m2idx = {calendar.month_name[i].lower()[:3]: i for i in range(1, 13)}

def sort_um_files(paths):
    paths_with_year_month = []
    for path in paths:
        # print(path)
        name = path.split('/')[-1]
        y = int(name.split('.')[1][2:6])
        midx = m2idx[name.split('.')[1][-3:]]
        paths_with_year_month.append((path, y, midx))
    
    sorted_paths = [x[0] for x in sorted(paths_with_year_month, key=lambda x: (x[1], x[2]))]
    return sorted_paths



In [4]:
#tmp_path = org_off_path
file_lists = glob.glob(basedir+'*.pd*.pp')
file_lists

['/gws/nopw/j04/mcs_prime/mmuetz/data/UM_sims/u-db150/init_test/db150a.pd1988oct.pp']

In [10]:
flag=1
nday=0
for file_path in sort_um_files(file_lists)[:1]:
    f=cf.read(file_path)
    dat=f[44]
    nday=dat.shape[0]+nday
    if flag == 1:
        dat_all=dat;
        flag = 0
    else:
        dat_all.data=dat_all.data+dat.data
    cf.close_files(file_path)
    print(file_path)
    # del f
    # del dat
prec_annual_on = dat_all.collapse('sum',axes='time')
prec_annual_on.data = prec_annual_on.data/nday*3600*24  # This looks suspicious! /(nday * 3600 * 24)?

/gws/nopw/j04/mcs_prime/mmuetz/data/UM_sims/u-db150/init_test/db150a.pd1988oct.pp


In [11]:
prec_annual_on

<CF Field: precipitation_flux(time(1), latitude(144), longitude(192)) m-2.kg.s-1>

In [25]:
#for i, field in enumerate(f):
#    print(i, field.identity())
print(f[43])
print(f[44])

Field: precipitation_flux (ncvar%UM_m01s05i216_vn1300)
------------------------------------------------------
Data            : precipitation_flux(time(720), latitude(144), longitude(192)) kg m-2 s-1
Cell methods    : time(720): mean
Dimension coords: time(720) = [1988-10-01 00:30:00, ..., 1988-10-30 23:30:00] 360_day
                : latitude(144) = [-89.375, ..., 89.375] degrees_north
                : longitude(192) = [0.9375, ..., 359.0625] degrees_east
Field: precipitation_flux (ncvar%UM_m01s05i216_vn1300)
------------------------------------------------------
Data            : precipitation_flux(time(30), latitude(144), longitude(192)) kg m-2 s-1
Cell methods    : time(30): mean
Dimension coords: time(30) = [1988-10-01 12:00:00, ..., 1988-10-30 12:00:00] 360_day
                : latitude(144) = [-89.375, ..., 89.375] degrees_north
                : longitude(192) = [0.9375, ..., 359.0625] degrees_east


In [26]:

#prec_obs_masked = prec_obs.where(prec_obs<0, np.nan)
#prec_on_masked = prec_annual_on.where(prec_obs<0, np.nan)
#prec_off_masked = prec_annual_off.where(prec_obs<0, np.nan)
#mip_mean_masked = mip_mean_prec.where(prec_obs.squeeze()<0, np.nan)

cfp.gopen()
cfp.cscale('plasma')
cfp.levs(min=0, max=15, step=1)
cfp.con(prec_annual_on, lines=False)

#flag_map = prec_annual_on.copy()
#flag_map.data = np.where((flag_map.data > mip_high) | (flag_map.data < mip_low), 1000, 0)
#cfp.stipple(f = flag_map, min=999, max=1001, size=60, color = 'grey', edgecolors='k', alpha=1)  # Swap indices

# Customize the plot style for AMS guidelines
#plt.gcf().set_size_inches(8, 6)  # Set the size of the figure
plt.xlabel('MCSP On: Rain Bias (mm day$^{-1}$)',fontsize=14)  # Set the x-axis label font size

# Save the plot using AMS-like settings
imagename='u-db150_init_test.png'
plt.savefig(imagename, dpi=300, bbox_inches='tight')

# Show the plot
#plt.show()
cfp.gclose()

In [ ]:
tmp_path = org_off_path
file_lists = glob.glob(tmp_path+'*.pp')

flag=1
nday=0
for file_path in sort_um_files(file_lists)[:12]:
    f=cf.read(file_path)
    dat=f[44]
    nday=dat.shape[0]+nday
    if flag == 1:
        dat_all=dat;
        flag = 0
    else:
        dat_all.data=dat_all.data+dat.data
    cf.close_files(file_path)
    print(file_path)
    del f
    del dat
prec_annual_off = dat_all.collapse('sum',axes='time')
prec_annual_off.data = prec_annual_off.data/nday*3600*24

In [ ]:
tmp_path = obs_path
file_lists = sorted(glob.glob(tmp_path+'*.HDF5'))

flag = 1;
for file_path in file_lists[:12]:
    # Open the IMERG HDF5 file in read mode
    file = h5py.File(file_path, 'r')

    # Read variables
    time_bnd = file['/Grid/time_bnds'][:]
    tlen = (time_bnd[0,1]-time_bnd[0,0])/3600

    if flag == 1:
        # Read the latitude and longitude datasets
        latitude = file['/Grid/lat'][:]
        longitude = file['/Grid/lon'][:]
        precipitation_rate = file['/Grid/precipitation'][:] * tlen
        ttotal = tlen
        flag = 0
    else:
        precipitation_rate = precipitation_rate + file['/Grid/precipitation'][:] * tlen
        ttotal = ttotal + tlen

    # Close the HDF5 file
    file.close()
    print(file_path)

precipitation_rate = precipitation_rate / ttotal

In [ ]:
source_grid

In [ ]:
# Regrid OBS
da = xr.DataArray(precipitation_rate, dims=('time', 'lon', 'lat'),
                  coords={'lat': latitude, 'lon': longitude})

# Define the model coordinate grid
model_latitude = prec_annual_off.coordinate('latitude').array # Model latitude values
model_longitude = prec_annual_off.coordinate('longitude').array  # Model longitude values

# Create source and destination grid coordinates using xarray DataArrays
source_grid = xr.Dataset({'latitude': da['lat'], 'longitude': da['lon']})
dest_grid = xr.Dataset({'latitude': model_latitude, 'longitude': model_longitude})

# Perform conservative regridding
regridder = xe.Regridder(source_grid, dest_grid, method='conservative')
regridded_da = regridder(da)

# Print the regridded precipitation rate data
print("Regridded precipitation rate data:")
print(regridded_da)

# Access the regridded precipitation rate values
regridded_precipitation = regridded_da.values

prec_obs = prec_annual_off.copy()
prec_obs.data = regridded_precipitation*24

In [ ]:
tmp_path = cmip_path
file_lists = sorted(glob.glob(tmp_path+'*.nc'))

flag = 1
for file_path in file_lists:
    f=cf.read(file_path)
    dat=f[0]
    dat.data[0:8]=0
    dat.data[68:72]=0
    #dat = dat.collapse('sum',axes='time')
    if flag == 1:
        mip_all=dat
        flag = 0
    else:
        mip_all=cf.aggregate([mip_all,dat])
    cf.close_files(file_path)
    print(file_path)
    del f
    del dat

i=0
for tmpdat in mip_all:
    tmpdat = tmpdat.collapse('sum',axes='time')
    mip_all[i] = tmpdat*3600*24/(12*5)
    i = i+1

In [ ]:
cmip_path

In [ ]:
# Define the model coordinate grid
model_latitude = prec_annual_off.coordinate('latitude').array # Model latitude values
model_longitude = prec_annual_off.coordinate('longitude').array  # Model longitude values

mip_latitude = mip_all[0].coordinate('latitude').array
mip_longitude = mip_all[0].coordinate('longitude').array

# Create source and destination grid coordinates using xarray DataArrays
source_grid = xr.Dataset({'latitude': mip_latitude, 'longitude': mip_longitude})
dest_grid = xr.Dataset({'latitude': model_latitude, 'longitude': model_longitude})

regridder = xe.Regridder(source_grid, dest_grid, method='conservative')

In [ ]:
# Regrid mip
prec_mip=np.empty((len(mip_all),model_latitude.shape[0],model_longitude.shape[0]),dtype=np.float32)
i=0
for tmpmip in mip_all:
    da = xr.DataArray(tmpmip, dims=('time', 'lat', 'lon'),
                      coords={'lat': mip_latitude, 'lon': mip_longitude})
    
    # Perform conservative regridding
    regridded_da = regridder(da)
    
    # Access the regridded precipitation rate values
    prec_mip[i] = regridded_da.values
    i = i + 1

In [ ]:
# Compute mip variation
mip_var=np.empty((model_latitude.shape[0],model_longitude.shape[0]),dtype=np.float32)
for i in np.arange (0,len(model_latitude)):
    for j in np.arange (0,len(model_longitude)):
        mip_var[i,j] = np.percentile(prec_mip[:,i,j],75) - np.percentile(prec_mip[:,i,j],25)

In [ ]:
# plot CMIP IQR
mip_var_prec=prec_annual_on.copy().squeeze()
mip_var_prec.data = mip_var

cfp.cscale('percent_11lev')
cfp.levs(min=0, max=5, step=0.5)
cfp.con(mip_var_prec, lines=False)

# Customize the plot style for AMS guidelines
plt.gcf().set_size_inches(8, 6)  # Set the size of the figure
plt.xlabel('CMIP6 Precipitation IQR (mm day$^{-1}$)',fontsize=14)  # Set the x-axis label font size

# Save the plot using AMS-like settings
imagename='cmip6_iqr.png'
plt.savefig(imagename, dpi=300, bbox_inches='tight')

# Show the plot
plt.close()

In [ ]:
# plot mip mean
mip_var=np.empty((model_latitude.shape[0],model_longitude.shape[0]),dtype=np.float32)
for i in np.arange (0,len(model_latitude)):
    for j in np.arange (0,len(model_longitude)):
        mip_var[i,j] = np.mean(prec_mip[:,i,j])

mip_mean_prec=prec_annual_on.copy().squeeze()
mip_mean_prec.data = mip_var

cfp.cscale('percent_11lev')
cfp.levs(min=0, max=15, step=1)
cfp.con(mip_mean_prec, lines=False)

# Customize the plot style for AMS guidelines
plt.gcf().set_size_inches(8, 6)  # Set the size of the figure
plt.xlabel('CMIP6 Mean Precipitation (mm day$^{-1}$)',fontsize=14)  # Set the x-axis label font size

# Save the plot using AMS-like settings
imagename='cmip6_prec_mean.png'
plt.savefig(imagename, dpi=300, bbox_inches='tight')

# Show the plot
plt.close()

In [ ]:
# Plot significance org on
mip_low=np.empty((model_latitude.shape[0],model_longitude.shape[0]),dtype=np.float32)
mip_high=np.empty((model_latitude.shape[0],model_longitude.shape[0]),dtype=np.float32)
for i in np.arange (0,len(model_latitude)):
    for j in np.arange (0,len(model_longitude)):
        mip_high[i,j] = np.percentile(prec_mip[:,i,j],95)
        mip_low[i,j] = np.percentile(prec_mip[:,i,j],5)

# masked
prec_obs_masked = prec_obs.where(prec_obs<0, np.nan)
prec_on_masked = prec_annual_on.where(prec_obs<0, np.nan)
prec_off_masked = prec_annual_off.where(prec_obs<0, np.nan)
mip_mean_masked = mip_mean_prec.where(prec_obs.squeeze()<0, np.nan)

cfp.gopen()
cfp.cscale('percent_11lev')
cfp.levs(min=0, max=15, step=1)
cfp.con(prec_annual_on, lines=False)

flag_map = prec_annual_on.copy()
flag_map.data = np.where((flag_map.data > mip_high) | (flag_map.data < mip_low), 1000, 0)
cfp.stipple(f = flag_map, min=999, max=1001, size=60, color = 'grey', edgecolors='k', alpha=1)  # Swap indices

# Customize the plot style for AMS guidelines
plt.gcf().set_size_inches(8, 6)  # Set the size of the figure
plt.xlabel('MCSP On: Rain Bias (mm day$^{-1}$)',fontsize=14)  # Set the x-axis label font size

# Save the plot using AMS-like settings
imagename='rain_on.png'
plt.savefig(imagename, dpi=300, bbox_inches='tight')

# Show the plot
cfp.gclose()

In [ ]:
# Plot significance org off
cfp.gopen()
cfp.cscale('percent_11lev')
cfp.levs(min=0, max=15, step=1)
cfp.con(prec_annual_off, lines=False)

flag_map = prec_annual_off.copy()
flag_map.data = np.where((flag_map.data > mip_high) | (flag_map.data < mip_low), 1000, 0)
cfp.stipple(f = flag_map, min=999, max=1001, size=60, color = 'grey', edgecolors='k', alpha=1)  # Swap indices

# Customize the plot style for AMS guidelines
plt.gcf().set_size_inches(8, 6)  # Set the size of the figure
plt.xlabel('MCSP Off: Rain Bias (mm day$^{-1}$)',fontsize=14)  # Set the x-axis label font size

# Save the plot using AMS-like settings
imagename='rain_off.png'
plt.savefig(imagename, dpi=300, bbox_inches='tight')

# Show the plot
cfp.gclose()

In [ ]:
# Plot cmip 6 bias
prec_mip_bias = mip_mean_prec - prec_obs_masked

cfp.gopen()
cfp.cscale('scale1')
cfp.levs(min=-5, max=5, step=1)
cfp.con(prec_mip_bias, lines=False)

# Customize the plot style for AMS guidelines
plt.gcf().set_size_inches(8, 6)  # Set the size of the figure
plt.xlabel('CMIP6: Rain Bias (mm day$^{-1}$)',fontsize=14)  # Set the x-axis label font size

# Save the plot using AMS-like settings
imagename='mip_bias.png'
plt.savefig(imagename, dpi=300, bbox_inches='tight')

# Show the plot
cfp.gclose()

In [ ]:
# Plot significance org on bias
prec_on_bias = prec_on_masked - prec_obs_masked

cfp.gopen()
cfp.cscale('scale1')
cfp.levs(min=-5, max=5, step=1)
cfp.con(prec_on_bias, lines=False)

flag_map = prec_annual_on.copy()
flag_map.data = np.where((flag_map.data > mip_high) | (flag_map.data < mip_low), 1000, 0)
cfp.stipple(f = flag_map, min=999, max=1001, size=60, color = 'grey', edgecolors='k', alpha=1)  # Swap indices

# Customize the plot style for AMS guidelines
plt.gcf().set_size_inches(8, 6)  # Set the size of the figure
plt.xlabel('MCSP On: Rain Bias (mm day$^{-1}$)',fontsize=14)  # Set the x-axis label font size

# Save the plot using AMS-like settings
imagename='rain_on_bias.png'
plt.savefig(imagename, dpi=300, bbox_inches='tight')

# Show the plot
cfp.gclose()

In [ ]:
# Plot significance org off bias
prec_off_bias = prec_off_masked - prec_obs_masked

cfp.gopen()
cfp.cscale('scale1')
cfp.levs(min=-5, max=5, step=1)
cfp.con(prec_off_bias, lines=False)

flag_map = prec_annual_off.copy()
flag_map.data = np.where((flag_map.data > mip_high) | (flag_map.data < mip_low), 1000, 0)
cfp.stipple(f = flag_map, min=999, max=1001, size=60, color = 'grey', edgecolors='k', alpha=1)  # Swap indices

# Customize the plot style for AMS guidelines
plt.gcf().set_size_inches(8, 6)  # Set the size of the figure
plt.xlabel('MCSP Off: Rain Bias (mm day$^{-1}$)',fontsize=14)  # Set the x-axis label font size

# Save the plot using AMS-like settings
imagename='rain_off_bias.png'
plt.savefig(imagename, dpi=300, bbox_inches='tight')

# Show the plot
cfp.gclose()

In [ ]:
prec_diff = prec_annual_on - prec_annual_off

cfp.gopen()
cfp.cscale('scale1')
cfp.levs(min=-5, max=5, step=1)
cfp.con(prec_diff, lines=False)

flag_map = prec_annual_on.copy()
flag_map.data = np.where((flag_map.data > mip_high) | (flag_map.data < mip_low), 1000, 0)
cfp.stipple(f = flag_map, min=999, max=1001, size=60, color = 'grey', edgecolors='k', alpha=1)  # Swap indices

# Customize the plot style for AMS guidelines
plt.gcf().set_size_inches(8, 6)  # Set the size of the figure
plt.xlabel('(c) = (b) - (a): Rain Difference (mm day$^{-1}$)',fontsize=14)  # Set the x-axis label font size

# Save the plot using AMS-like settings
imagename='rain_diff.png'
plt.savefig(imagename, dpi=300, bbox_inches='tight')

# Show the plot
cfp.gclose()

In [ ]:
bias_diff = prec_on_bias.copy()
bias_diff.data = np.abs(prec_on_bias.array) - np.abs(prec_off_bias.array)

cfp.gopen()
cfp.cscale('scale1')
cfp.levs(min=-5, max=5, step=1)
cfp.con(bias_diff, lines=False)

flag_map = prec_annual_on.copy()
flag_map.data = np.where((flag_map.data > mip_high) | (flag_map.data < mip_low), 1000, 0)
cfp.stipple(f = flag_map, min=999, max=1001, size=60, color = 'grey', edgecolors='k', alpha=1)  # Swap indices

# Customize the plot style for AMS guidelines
plt.gcf().set_size_inches(8, 6)  # Set the size of the figure
plt.xlabel('MCSP On - Off: Bias Change (mm day$^{-1}$)',fontsize=14)  # Set the x-axis label font size

# Save the plot using AMS-like settings
imagename='rain_bias_change.png'
plt.savefig(imagename, dpi=300, bbox_inches='tight')

# Show the plot
cfp.gclose()

In [ ]:
df = pd.DataFrame({'arr1': prec_obs_masked.array.flatten(), 'arr2': prec_off_masked.array.flatten()})
corr_coef = df['arr1'].corr(df['arr2'])
print('obs vs off:', corr_coef)

df = pd.DataFrame({'arr1': prec_obs_masked.array.flatten(), 'arr2': prec_on_masked.array.flatten()})
corr_coef = df['arr1'].corr(df['arr2'])
print('obs vs on:', corr_coef)

df = pd.DataFrame({'arr1': prec_obs_masked.array.flatten(), 'arr2': mip_mean_masked.array.flatten()})
corr_coef = df['arr1'].corr(df['arr2'])
print('obs vs cmip:', corr_coef)

In [ ]:
# Plot obs
cfp.gopen()
cfp.cscale('percent_11lev')
cfp.levs(min=0, max=15, step=1)
cfp.con(prec_obs, lines=False)

# Customize the plot style for AMS guidelines
plt.gcf().set_size_inches(8, 6)  # Set the size of the figure
plt.xlabel('IMERG: Rain Rate (mm day$^{-1}$)',fontsize=14)  # Set the x-axis label font size

# Save the plot using AMS-like settings
imagename='rain_obs.png'
plt.savefig(imagename, dpi=300, bbox_inches='tight')

# Show the plot
cfp.gclose()